In [17]:
import pandas as pd

In [18]:
ratings = pd.read_csv('co-ratings.csv')
ratings.head()

,users,ratings,route_id,name,grade,type
0,8dfba0f922c3d59ebd4cfb9e1d05c365591d1c38,4.0,119270673,Wild Arete,5.12a/b,"{'sport': True, 'tr': True}"
1,bec34299bb47603530d9e3e012530fd75ab9dd8e,4.0,119270673,Wild Arete,5.12a/b,"{'sport': True, 'tr': True}"
2,ac234798a91219e5667ccbfea2ef7dc90d5c05e8,4.0,107142013,Zebra,5.8+,{'sport': True}
3,95dc46e5b540a3a21df76aee087268510b646659,4.0,107142013,Zebra,5.8+,{'sport': True}
4,5a8f95e578f056ad023a60df69b19034d2538178,4.0,107142013,Zebra,5.8+,{'sport': True}


In [19]:
ratings.groupby(['route_id', 'name'])['ratings'].count().reset_index(name="count")

,route_id,name,count
0,105747119,Backside Flows,2
1,105747123,Main Flow,7
2,105747127,The Bouldering Area,6
3,105747131,Middle Flow,14
4,105747135,The Ramp,13
...,...,...,...
18727,119398005,Route 1,1
18728,119398098,Route 2,1
18729,119401183,North Ridge (Kit Carson),5
18730,119402530,Shade Tree Mechanic,3


In [20]:
popular = ratings.groupby(['route_id', 'name'])['ratings'].count().reset_index(name="count").sort_values(by='count', ascending=False)
popular.head(10)

,route_id,name,count
246,105748047,East Face (Standard),1850
368,105748490,The Bastille Crack,1842
1346,105752023,Freeway,1598
842,105750106,Rewritten,1345
304,105748268,Direct Route,1080
423,105748657,The Yellow Spur,1035
330,105748361,Wind Ridge,1001
340,105748391,Calypso,961
9146,106701638,Playin' Hooky,872
743,105749785,Animation,853


In [13]:
from surprise import Dataset, Reader, accuracy, KNNBasic
from surprise.model_selection import train_test_split, KFold

reader = Reader(rating_scale=(0, 4))
data = Dataset.load_from_df(ratings[['users', 'route_id', 'ratings']], reader)

sim_options = {'name': 'cosine', 'user_based': True, 'min_support': 4}

algo = KNNBasic(sim_options=sim_options)
kf = KFold(n_splits=5)

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)


Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.6712
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.6668
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.6705
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.6698
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.6682


In [21]:
climb_name = "Freeway"

route_id = ratings[ratings['name'] == climb_name]['route_id'].iloc[1]

print("People who climbed '{}' also climbed:".format(climb_name))

predictions = algo.get_neighbors(trainset.to_inner_iid(route_id), 50)

print(predictions)

People who climbed 'Freeway' also climbed:
[83, 175, 201, 239, 349, 418, 504, 517, 532, 604, 624, 702, 719, 752, 758, 788, 864, 915, 940, 1065, 1067, 1115, 1129, 1233, 1241, 1300, 1367, 1384, 1391, 1398, 1418, 1423, 1610, 1613, 1636, 1755, 1759, 1801, 1880, 1902, 1942, 1949, 1952, 1967, 1979, 2033, 2047, 2198, 2204, 2215]


In [22]:
import numpy as np
recs = map(lambda x: trainset.to_raw_iid(x), np.asarray(predictions))

results = ratings[ratings['route_id'].isin(recs)]

r = results.pivot_table(index=['name', 'route_id', 'type', 'grade'], aggfunc=[np.mean, np.median, np.size], values='ratings')

print(r)

                                                                                    mean  \
                                                                                 ratings   
name                         route_id  type                            grade               
99% Pure                     105751918 {'trad': True}                  5.9      2.000000   
Anguish and Fear             106126236 {'sport': True}                 5.10a    2.693694   
Apes in Estrus               105760140 {'sport': True}                 5.10a    2.421053   
Bitty Buttress               105748265 {'trad': True}                  5.8+     3.314655   
Book  of Numbers             105763329 {'trad': True, 'sport': True}   5.12c/d  3.111111   
Borgoff's Blunder            105758739 {'trad': True, 'sport': True}   5.10a    2.181818   
Chimney Crack                108131535 {'trad': True, 'tr': True}      5.6      1.857143   
Crack a Beer                 106250633 {'trad': True}                  5.10+    

/var/folders/zc/f4mvm36553n45t7v2cf45kr00000gn/T/ipykernel_65372/4230413558.py:6: FutureWarning: The provided callable <function mean at 0x105f8f880> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  r = results.pivot_table(index=['name', 'route_id', 'type', 'grade'], aggfunc=[np.mean, np.median, np.size], values='ratings')
/var/folders/zc/f4mvm36553n45t7v2cf45kr00000gn/T/ipykernel_65372/4230413558.py:6: FutureWarning: The provided callable <function median at 0x10a0e7c40> is currently using DataFrameGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  r = results.pivot_table(index=['name', 'route_id', 'type', 'grade'], aggfunc=[np.mean, np.median, np.size], values='ratings')
